In [ ]:
import pandas as pd
import numpy as np
import csv
import ast

: 

: 

In [ ]:
data = pd.read_csv("standardized.csv")
data = data.dropna(subset=["production_countries", "weighted_average_popularity"])
country_dict = {}

for index, row in data.iterrows():
    countries = row["production_countries"]
    popularity = row["weighted_average_popularity"]
    countries = ast.literal_eval(countries)
    
    for i in range(len(countries)):
        country = countries[i]
        if country not in country_dict:
            country_dict[country] = [popularity]
        else:
            country_dict[country].append(popularity)
        i += 1

country_popularity_dict = {}
for key, value in country_dict.items():
    average = sum(value) / len(value)
    country_popularity_dict[key] = round(average, 2)

df = pd.DataFrame(list(country_popularity_dict.items()), columns = ["country", "weighted_average_popularity"])

df.to_csv("country.csv", index = False)


: 

In [ ]:
from sklearn.feature_selection import mutual_info_classif

data = pd.read_csv("country.csv")
data = data.dropna(subset = ["country", "weighted_average_popularity"])

data["range"] = pd.cut(data["weighted_average_popularity"], bins=[0, 0.5, 1], labels=['<=0.5', '>0.5'])
countries = data[["country"]].copy()
popularity = data["range"]

countries["country"] = countries["country"].astype("category")

mi = mutual_info_classif(X=countries, y=popularity, discrete_features=True)
print(mi)


: 

The mutual information between country and popularity is 0.8, which shows there is a relationship between them